In [1]:
import math
import torch
import torch.nn.functional as F

In [14]:
num_steps = 8

In [15]:
for k in range(0, num_steps-4):
    print(k)

print(num_steps-4)
print(num_steps-3)
print(num_steps-2)
print(num_steps-1)

0
1
2
3
4
5
6
7


In [341]:
def unfolded(X):
    Xa = X
    num_steps = int(math.log2(X.size(0)))

    for k in range(num_steps-6):
        T = 2*(Xa.size(0)//2)

        Xa = Xa[:T].view(T//2, 2)
        Xa[:, 1].add_(Xa[:, 0])
        Xa = Xa[:, 1]

    # num_steps-6
    T = 2*(Xa.size(0) // 2)
    Xa = Xa[:T].view(T//2, 2)
    Xa[:, 1].add_(Xa[:, 0])
    Xa = Xa[:, 1]

    # num_steps-5
    T = 2*(Xa.size(0) // 2)
    Xa = Xa[:T].view(T//2, 2)
    Xa[:, 1].add_(Xa[:, 0])
    Xa = Xa[:, 1]

    # num_steps-4
    T = 2*(Xa.size(0) // 2)
    Xa = Xa[:T].view(T//2, 2)
    Xa[:, 1].add_(Xa[:, 0])
    Xa = Xa[:, 1]

    # num_steps-3
    T = 2*(Xa.size(0) // 2)
    Xa = Xa[:T].view(T//2, 2)
    Xa[:, 1].add_(Xa[:, 0])
    Xa = Xa[:, 1]

    # num_steps-2
    T = 2*(Xa.size(0) // 2)
    Xa = Xa[:T].view(T//2, 2)
    Xa[:, 1].add_(Xa[:, 0])
    Xa = Xa[:, 1]

    # num_steps-1
    T = 2*(Xa.size(0) // 2)
    Xa = Xa[:T].view(T//2, 2)
    Xa[:, 1].add_(Xa[:, 0])
    Xa = Xa[:, 1]

def unfolded_3(X):
    Xa = X
    num_steps = int(math.log2(X.size(0)))

    for k in range(num_steps-3):
        T = 2*(Xa.size(0)//2)

        Xa = Xa[:T].view(T//2, 2)
        Xa[:, 1].add_(Xa[:, 0])
        Xa = Xa[:, 1]
    
    Xa[1].add_(Xa[0])
    Xa[3].add_(Xa[2])
    Xa[5].add_(Xa[4])
    Xa[7].add_(Xa[6])

    Xa[3].add_(Xa[1])
    Xa[7].add_(Xa[5])

    Xa[7].add_(Xa[3])

def unfolded_4(X):
    Xa = X
    num_steps = int(math.log2(X.size(0)))

    for k in range(num_steps-3):
        T = 2*(Xa.size(0)//2)

        Xa = Xa[:T].view(T//2, 2)
        Xa[:, 1].add_(Xa[:, 0])
        Xa = Xa[:, 1]
    
    Xa[1].add_(Xa[0])
    Xa[3].add_(Xa[1]+Xa[2])
    Xa[5].add_(Xa[4])
    Xa[7].add_(Xa[3]+X[5]+X[6])

### BEST ###
def unfolded_2(X):
    Xa = X
    num_steps = int(math.log2(X.size(0)))

    for _ in range(num_steps-2):
        T = 2*(Xa.size(0)//2)

        Xa = Xa[:T].view(T//2, 2)
        Xa[:, 1].add_(Xa[:, 0])
        Xa = Xa[:, 1]
    
    # the last 2 steps are unfolded/done manually (up to 7 nodes)
    T = 2 * (Xa.size(0) // 2)
    if T == 6:
        Xa[1].add_(Xa[0])
        Xa[3].add_(Xa[2].add(Xa[1]))
        Xa[5].add_(Xa[4])
    elif T == 4:
        Xa[1].add_(Xa[0])
        Xa[3].add_(Xa[2].add(Xa[1]))
    elif T == 3:
        Xa[1].add_(Xa[0])
    elif T == 2:
        Xa[1].add_(Xa[0])

def normal(X):
    Xa = X
    num_steps = int(math.log2(X.size(0)))

    for k in range(num_steps):
        T = 2*(Xa.size(0)//2)

        Xa = Xa[:T].view(T//2, 2)
        Xa[:, 1].add_(Xa[:, 0])
        Xa = Xa[:, 1]

In [342]:
L = 8
X = torch.arange(L).to("cuda")
X

tensor([0, 1, 2, 3, 4, 5, 6, 7], device='cuda:0')

In [343]:
X = torch.arange(L).to("cuda")
normal(X)
X

tensor([ 0,  1,  2,  6,  4,  9,  6, 28], device='cuda:0')

In [344]:
X = torch.arange(L).to("cuda")
unfolded_2(X)
X

tensor([ 0,  1,  2,  6,  4,  9,  6, 28], device='cuda:0')

In [204]:
X = torch.arange(L).to("cuda")
unfolded_3(X)
X

tensor([ 0,  1,  2,  6,  4,  9,  6, 28], device='cuda:0')

In [205]:
X = torch.arange(L).to("cuda")
unfolded_4(X)
X

tensor([ 0,  1,  2,  6,  4,  9,  6, 28], device='cuda:0')

In [348]:
L = 512
X = torch.arange(L).to("cuda")

In [349]:
%timeit normal(X)

198 µs ± 1.81 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [350]:
%timeit unfolded_2(X)

181 µs ± 2.09 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [219]:
%timeit unfolded_3(X)

226 µs ± 6.26 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [220]:
%timeit unfolded_4(X)

225 µs ± 4.69 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [705]:
# BEST
# unfold up sweep (2 steps) et down sweep (2 steps)
def unfolded_2(X):
    Xa = X
    num_steps = int(math.log2(X.size(0)))

    # up sweep (unfolded)
    for _ in range(num_steps-2):
        T = 2*(Xa.size(0)//2)

        Xa = Xa[:T].view(T//2, 2)
        Xa[:, 1].add_(Xa[:, 0])
        Xa = Xa[:, 1]
    
    # the last 2 steps are unfolded/done manually (up to 7 nodes)
    T = 2 * (Xa.size(0) // 2)
    if T == 6:
        Xa[1].add_(Xa[0])
        Xa[3].add_(Xa[2].add(Xa[1]))
        Xa[5].add_(Xa[4])
    elif T == 4:
        Xa[1].add_(Xa[0])
        Xa[3].add_(Xa[2].add(Xa[1]))
    elif T == 3:
        Xa[1].add_(Xa[0])
    elif T == 2:
        Xa[1].add_(Xa[0])

    # down sweep (unfolded)
    Xa = X[2**(num_steps-1)-1:L:2**(num_steps-1)]
    T = 2 * (Xa.size(0) // 2)

    if T < Xa.size(0):
        Xa[2].add_(Xa[1])

    Xa = X[2**(num_steps-2)-1:L:2**(num_steps-2)]
    T = 2 * (Xa.size(0) // 2)

    #if T < Xa.size(0):
    #    Xa[4].add_(Xa[3])

    Xa[4].add_(Xa[3])
    Xa[2].add_(Xa[1])

    for k in range(num_steps-3, -1, -1):
        Xa = X[2**k-1:L:2**k]

        T = 2 * (Xa.size(0) // 2)

        if T < Xa.size(0):
            Xa[:, -1].add_(Xa[:, -2])

        Xa = Xa[:T].view(T//2, 2)

        Xa[1:, 0].add_(Xa[:-1, 1])

def unfolded_2_with_flag(X):
    Xa = X
    num_steps = int(math.log2(X.size(0)))

    flagstep = -1

    # up sweep (unfolded)
    for k in range(num_steps-2):
        T = 2*(Xa.size(0)//2)
        
        if T < Xa.size(0):
            flagstep = k

        Xa = Xa[:T].view(T//2, 2)
        Xa[:, 1].add_(Xa[:, 0])
        Xa = Xa[:, 1]
    
    # the last 2 steps are unfolded/done manually (up to 7 nodes)
    T = 2 * (Xa.size(0) // 2)
    
    if Xa.size(0) == 7:
        flagstep = num_steps - 2
    elif Xa.size(0) == 6:
        flagstep = num_steps - 1
    elif Xa.size(0) == 5:
        flagstep = num_steps - 2
    elif Xa.size(0) == 3:
        flagstep = num_steps - 1

    if T == 6:
        Xa[1].add_(Xa[0])
        Xa[3].add_(Xa[2].add(Xa[1]))
        Xa[5].add_(Xa[4])
    elif T == 4:
        Xa[1].add_(Xa[0])
        Xa[3].add_(Xa[2].add(Xa[1]))
    elif T == 3:
        Xa[1].add_(Xa[0])
    elif T == 2:
        Xa[1].add_(Xa[0])

    # down sweep (unfolded)
    Xa = X[2**(num_steps-2)-1:L:2**(num_steps-2)]
    if flagstep == num_steps - 1:
        Xa[5].add_(Xa[3])
        Xa[4].add_(Xa[3])
    elif flagstep == num_steps - 2:
        print("hehe")
        Xa[4].add_(Xa[3])

    Xa[2].add_(Xa[1])

    for k in range(num_steps-3, -1, -1):
        Xa = X[2**k-1:L:2**k]

        T = 2 * (Xa.size(0) // 2)

        if k == flagstep:
            Xa[:, -1].add_(Xa[:, -2])

        Xa = Xa[:T].view(T//2, 2)

        Xa[1:, 0].add_(Xa[:-1, 1])

# downsweep unfolded sur 3 steps
def unfolded_22(X):
    Xa = X
    num_steps = int(math.log2(X.size(0)))

    # up sweep (unfolded)
    for _ in range(num_steps-2):
        T = 2*(Xa.size(0)//2)

        Xa = Xa[:T].view(T//2, 2)
        Xa[:, 1].add_(Xa[:, 0])
        Xa = Xa[:, 1]
    
    # the last 2 steps are unfolded/done manually (up to 7 nodes)
    T = 2 * (Xa.size(0) // 2)
    if T == 6:
        Xa[1].add_(Xa[0])
        Xa[3].add_(Xa[2].add(Xa[1]))
        Xa[5].add_(Xa[4])
    elif T == 4:
        Xa[1].add_(Xa[0])
        Xa[3].add_(Xa[2].add(Xa[1]))
    elif T == 3:
        Xa[1].add_(Xa[0])
    elif T == 2:
        Xa[1].add_(Xa[0])

    # down sweep (unfolded)
    Xa = X[2**(num_steps-3)-1:L:2**(num_steps-3)] # -2
    Xa[5].add_(Xa[3])

    Xa[4].add_(Xa[3])
    Xa[6].add_(Xa[5])

    for k in range(num_steps-4, -1, -1): # -3
        Xa = X[2**k-1:L:2**k]

        T = 2 * (Xa.size(0) // 2)

        if T < Xa.size(0):
            Xa[:, -1].add_(Xa[:, -2])

        Xa = Xa[:T].view(T//2, 2)

        Xa[1:, 0].add_(Xa[:-1, 1])

def normal(X):
    Xa = X
    num_steps = int(math.log2(X.size(0)))

    # up sweep
    for _ in range(num_steps):
        T = 2*(Xa.size(0)//2)

        Xa = Xa[:T].view(T//2, 2)
        Xa[:, 1].add_(Xa[:, 0])
        Xa = Xa[:, 1]

    # down sweep
    for k in range(num_steps-1, -1, -1):
        Xa = X[2**k-1:L:2**k]

        T = 2 * (Xa.size(0) // 2)

        if T < Xa.size(0):
            Xa[-1].add_(Xa[-2])

        Xa = Xa[:T].view(T//2, 2)

        Xa[1:, 0].add_(Xa[:-1, 1])

In [706]:
L = 7

In [707]:
X = torch.arange(L).to("cuda")
normal(X)
X

tensor([ 0,  1,  3,  6, 10, 15, 21], device='cuda:0')

In [708]:
X = torch.arange(L).to("cuda")
unfolded_2(X)
X

tensor([ 0,  1,  3,  6, 10, 15,  6], device='cuda:0')

In [709]:
X = torch.arange(L).to("cuda")
unfolded_2_with_flag(X)
X

hehe


tensor([ 0,  1,  3,  6, 10,  9,  6], device='cuda:0')

In [7]:
len = 64
a = torch.ones(len)

In [8]:
def pad(a):
    len_npo2 = 2**math.ceil(math.log2(len))
    pad_tuple = (0, len_npo2-len)
    return F.pad(a, pad_tuple, "constant", 0)

In [9]:
%timeit pad(a)

2.45 µs ± 50.1 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [10]:
def pad_custom(a):
    len_npo2 = 2**math.ceil(math.log2(len))

    zeros = torch.zeros(len_npo2)
    zeros[:a.size(0)] = a
    return zeros

In [11]:
%timeit pad_custom(a)

4.07 µs ± 244 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [15]:
2**math.ceil(math.log2(1))

1

In [438]:
X = torch.arange(L).to("cuda")
unfolded_22(X)
X

tensor([ 0,  1,  2,  6, 10, 15, 21, 28], device='cuda:0')

In [497]:
L = 2048

In [500]:
X = torch.arange(L).to("cuda")
%timeit unfolded_2(X)

431 µs ± 5.13 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [501]:
X = torch.arange(L).to("cuda")
%timeit normal(X)

495 µs ± 20.7 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [64]:
def npo2_unfolded(X):
    Xa = X
    num_steps = int(math.log2(X.size(0)))

    # up sweep (last 2 steps unfolded)
    for k in range(num_steps-2):
        Xa = Xa.view(Xa.size(0) // 2, 2)
        Xa[:, 1].add_(Xa[:, 0])
        Xa = Xa[:, 1]
    
    # (at this stage, we can have 4, 2 or 1 nodes) 
    if Xa.size(0) == 4:
        Xa[1].add_(Xa[0])
        Xa[3].add_(Xa[2].add(Xa[1]))
    elif Xa.size(0) == 2:
        Xa[1].add_(Xa[0])

    # down sweep (first 2 steps unfolded)
    Xa = X[2**(num_steps-2)-1:L:2**(num_steps-2)]
    Xa[2].add_(Xa[1])

    for k in range(num_steps-3, -1, -1):
        Xa = X[2**k-1:L:2**k]
        Xa = Xa.view(Xa.size(0) // 2, 2)
        Xa[1:, 0].add_(Xa[:-1, 1])

def npo2_normal(X):
    Xa = X
    num_steps = int(math.log2(X.size(0)))

    # up sweep
    for _ in range(num_steps):
        T = 2*(Xa.size(0)//2)

        Xa = Xa[:T].view(T//2, 2)
        Xa[:, 1].add_(Xa[:, 0])
        Xa = Xa[:, 1]

    # down sweep
    for k in range(num_steps-1, -1, -1):
        Xa = X[2**k-1:L:2**k]

        T = 2 * (Xa.size(0) // 2)

        Xa = Xa[:T].view(T//2, 2)

        Xa[1:, 0].add_(Xa[:-1, 1])

In [65]:
L = 1024

In [66]:
X = torch.arange(L).to("cuda")
npo2_normal(X)
X

tensor([     0,      1,      3,  ..., 521731, 522753, 523776], device='cuda:0')

In [67]:
X = torch.arange(L).to("cuda")
npo2_unfolded(X)
torch.allclose(X, torch.cumsum(torch.arange(L).to("cuda"), 0))

True

In [71]:
X = torch.arange(L).to("cuda")
%timeit npo2_normal(X)

458 µs ± 9.98 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [70]:
X = torch.arange(L).to("cuda")
%timeit npo2_unfolded(X)

387 µs ± 11 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [72]:
X = torch.randn(5, 33, 16, 16) # (B, L, D, N)

In [74]:
len_npo2 = 2**math.ceil(math.log2(X.size(1)))
pad_tuple = (0, 0, 0, 0, len_npo2 - X.size(1), 0)
F.pad(X, pad_tuple, "constant", 0).shape

torch.Size([5, 64, 16, 16])

In [75]:
def pad_npo2(X):
    # X : (B, L, D, N)

    # Y : (B, npo2(L), D, N)

    len_npo2 = 2**math.ceil(math.log2(X.size(1)))
    pad_tuple = (0, 0, 0, 0, len_npo2 - X.size(1), 0)
    return F.pad(X, pad_tuple, "constant", 0)

In [82]:
%timeit pad_npo2(torch.randn(64, 33, 32, 16))

5.65 ms ± 70.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [89]:
g = torch.randn(64, 64, 32, 16).to("cuda")

In [90]:
%timeit pad_npo2(g)

49.3 µs ± 475 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [91]:
%timeit g.clone()

49 µs ± 165 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
